# Danny Silvestre Assignment 3
Computational Foundations of AI
12/3/2021
z23431842
Professor Michael DeGiorgio

In [ ]:
### This Module prepares the data and initializes an organized, scaled dataset. (CreditData)
# importing the module
import pandas as pd
import numpy as np
# read specific columns of csv file using Pandas 
Features = []
Labels = []
df = pd.read_csv("TrainingData_N183_p10.csv")
#Feature Extraction
featureExtract = pd.read_csv("TrainingData_N183_p10.csv",usecols = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"])

quantitative = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"]


for featureName in featureExtract:
  featureCol = []
  for featureValues in featureExtract[featureName]:
    featureCol.append(featureValues)
  Features.append(featureCol)

#Feature Normalization and Scaling
ScaledFeatures = []
for featureType in Features:
  print(featureType)
  scaledFeatureType = []
  mean = np.mean(featureType)
  std = np.std(featureType)
  #Scale each feature value
  for val in featureType:
    newVal = (val - mean) / std
    scaledFeatureType.append(newVal)
  ScaledFeatures.append(scaledFeatureType)
#Labels in list.
for label in df['Ancestry']:
  Labels.append(label)


print("Features")
print(Features)
print("Scaled Features")
print(np.array(ScaledFeatures).shape)
print("Training Data")
print(featureExtract)

X = [[ScaledFeatures[0][i]] + [ScaledFeatures[1][i]] + [ScaledFeatures[2][i]] + [ScaledFeatures[3][i]] + [ScaledFeatures[4][i]] + [ScaledFeatures[5][i]] + [ScaledFeatures[6][i]] + [ScaledFeatures[7][i]] + [ScaledFeatures[8][i]] + [ScaledFeatures[9][i] ]for i in range(len(ScaledFeatures[0]))]

F = np.array(X)
print(F.shape)
print(X[0])
# Centering labels
#Labels -= np.mean(Labels)
y = Labels

#𝐾 =5 
#continental  ancestries  (African,  European,  East  Asian,  Oceanian,  or  Native  American).
cat = ["African","European","EastAsian", "Oceanian", "NativeAmerican"] 
def one_hot_encoding(labels,categories):
  newArray = []
  for item in labels:
    tmp = np.zeros(len(categories))
    for x in range(len(categories)):
      if item == categories[x]:
        tmp[x] = 1
    newArray.append(tmp)
  return newArray
y = one_hot_encoding(y,cat)
print(np.array(y).shape)
#print(df)
#printing the first instance of credit data

In [ ]:
# Centering labels
#Labels -= np.mean(Labels)
#y = Labels
#y

In [ ]:
import sys

from random import seed
from random import randrange
# Split a dataset into k folds
def cross_validation_split(x,y, folds):
	features = []
	labels = []
	x_copy = list(x)
	y_copy = list(y)
	fold_size = int(len(x) / folds)
	for i in range(folds):
		x_fold = []
		y_fold = []
		while len(x_fold) < fold_size:
			index = randrange(len(x_copy))
			x_fold.append(x_copy.pop(index))
			y_fold.append(y_copy.pop(index))
		features.append(x_fold)
		labels.append(y_fold)
	return features,labels
 

In [ ]:
import time
from progressbar import ProgressBar
class RidgeRegression() :
      
    def __init__( self, learning_rate, iterations, tuning,verbose ) :
      if verbose == 1:
        print("Initializing Model with learning rate of " + str(learning_rate))
        print("Number of Iterations " + str(iterations))        
        print("Tuning Parameter " + str(tuning)) 
      self.learning_rate = learning_rate
      self.tuning = tuning
      self.iterations = iterations
      self.verbose = verbose 
    # This is going to be used to train the models           
    def fit( self, X, Y ) :
          
        # This is the number of training examples and number of features stored in the class.       
        self.numExamples, self.numFeatures = X.shape
        if (self.verbose >= 1):
          print("Calculating number of training examples...")
          print(self.numExamples)
          print("Calculating number of features...")
          print(self.numFeatures)
          if (self.verbose == 2):
            print("Current Weights")
            print(np.rand( self.numFeatures ))
        # step 3 inializing random weights     
        self.W = np.random.rand(self.numFeatures)
        # Initializing the output layer because there are 5 classes. One for each type of ancestry
        self.outputLayer = np.random.rand(5)
        self.b = 0        
        self.X = X        
        self.Y = Y
        #This gives it time so it doesnt overwrite the other print functions
        time.sleep(0.5)
        # GDL
        #Progress bar is used so that the console stays organized rather than printing out multiple lines.
        pbar = ProgressBar()
        print("\n iterations")
        for i in pbar(range(self.iterations )):        
            self.update_weights()            
        return self
      
    # Updates weights for GDL
    def update_weights( self ) :        
        Y_pred = self.predict( self.X )

        # calculating the gradients  
        #print("self.x.t") 
        #print(self.X.T) 
        #Change in the weights  
        totalLoss = self.Y - Y_pred
        class1Loss = [totalLoss[i][0] for i in range(len(self.X))]
        class2Loss = [totalLoss[i][1] for i in range(len(self.X))]
        class3Loss = [totalLoss[i][2] for i in range(len(self.X))]
        class4Loss = [totalLoss[i][3] for i in range(len(self.X))]
        class5Loss = [totalLoss[i][4] for i in range(len(self.X))]
        totalLoss = [class1Loss,class2Loss,class3Loss,class4Loss,class5Loss]
        dWEnd = 0
        dBEnd = 0
        for loss in totalLoss:
          dW = ( - ( 2 * ( self.X.T ).dot( loss) ) +               
               ( 2 * self.tuning * self.W ) ) / self.numExamples 
        #change in b     
          db = - 2 * np.sum( loss ) / self.numExamples
          dWEnd += dW
          dBEnd += db
        dW = dWEnd
        db = dBEnd
        # update weights    
        if (verbose == 2):
          print("Updating weights...")
        self.W = self.W - self.learning_rate * dW 
        self.b = self.b - self.learning_rate * db     
        if (verbose == 2 ):
          print("new weights: ")
          print(self.W)
          print("new b value: ")
          print(self.b)      
        return self
      
    
    def predict( self, X ) :
      predictions = []
      for ex in X:
        prediction = [ex.dot(self.W) * self.outputLayer[0],   ex.dot(self.W) * self.outputLayer[0],ex.dot(self.W) * self.outputLayer[0],ex.dot(self.W) * self.outputLayer[0],ex.dot(self.W) * self.outputLayer[0] ]
        predictions.append(prediction)
      return predictions
      #return X.dot( self.W ) 


In [ ]:
#9 Features plot of coefficients.
Tuning_parameters = [10**-2,10**-1,10**0,10**1,10**2,10**3,10**4]
lr = 10**-5
iteration = 10**4
verbose = 0
coef = []
#Africancoef = []
#Europeancoef = []
#EastAsiancoef = []
#Oceaniancoef = []
#NativeAmericancoef = []
tX = np.array(X)
#African = [y[i][0] for i in range(len(y))]
#European = [y[i][1] for i in range(len(y))]
#EastAsian = [y[i][2] for i in range(len(y))]
#Oceanian = [y[i][3] for i in range(len(y))]
#NativeAmerican = [y[i][4] for i in range(len(y))]

tY = np.array(y)
#tY = [African,European,EastAsian,Oceanian,NativeAmerican]
#tY = np.array(tY)
for tuningParameter in Tuning_parameters:
  ridge = RidgeRegression(lr,iteration,tuningParameter,verbose)
  ridge.fit(tX,tY)
  coef.append(ridge.W)
print(coef)

In [ ]:
#Deliverable 1
import matplotlib.pyplot as plt
pltfeatures = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"]
PC1= []
PC2 = []
PC3 = []
PC4 = []
PC5 = []
PC6 = []
PC7 = []
PC8 = []
PC9 = []
PC10 = []
for i in range(0,7):
  PC1.append(coef[i][0])
  PC2.append(coef[i][1])
  PC3.append(coef[i][2])
  PC4.append(coef[i][3])
  PC5.append(coef[i][4])
  PC6.append(coef[i][5])
  PC7.append(coef[i][6])
  PC8.append(coef[i][7])
  PC9.append(coef[i][8])
  PC10.append(coef[i][9])

fig, ax = plt.subplots()
plt.xscale('log')
ax.plot(Tuning_parameters,PC1,label="PC1")
ax.plot(Tuning_parameters,PC2,label="PC2")
ax.plot(Tuning_parameters,PC3,label="PC3")
ax.plot(Tuning_parameters,PC4,label="PC4")
ax.plot(Tuning_parameters,PC5,label="PC5")
ax.plot(Tuning_parameters,PC6,label="PC6")
ax.plot(Tuning_parameters,PC7,label="PC7")
ax.plot(Tuning_parameters,PC8,label="PC8")
ax.plot(Tuning_parameters,PC9,label="PC9")
ax.plot(Tuning_parameters,PC10,label="PC10")
ax.legend(loc = 'upper left')
plt.show()


In [ ]:
#Training the model using Cross Validation
seed(1)
Tuning_parameters = [10**-2,10**-1,10**0,10**1,10**2,10**3,10**4]
lr = 10**-5
iteration = 10**4
fold_num = 5
verbose = 1
ft,lb= cross_validation_split(X,y,5)

mseResults = []
for tuningParameter in Tuning_parameters:
  model = RidgeRegression(lr,iteration,tuningParameter,verbose)
  foldMSE = []
  for i in range(0,fold_num):
    print("***************************************")
    print("Fold " + str(i) + "/" + str(fold_num))
    print("Tuning parameter " + str(tuningParameter) )
    print("***************************************")
    valx = ft[i]
    valy = lb[i]
    trainx = []
    trainy = []
    for x in range (0,fold_num):
      if x != i:
        trainx += ft[x]
        trainy += lb[x]
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    valx = np.array(valx)
    valy = np.array(valy)
    #print(len(trainx))
    #print(len(trainy))
    #print(len(valx))
    #print(len(valy))
    model.fit(trainx,trainy)
    prediction = model.predict(valx)
    n = len(valy)
    #print("error")
    #print(valy - prediction)
    squaredError = ((valy - prediction) ** 2)
    error = np.mean(squaredError)
    #calculate test set accuracy here
    foldMSE.append(error)
  print(foldMSE)
  meanFoldMSE = np.mean(foldMSE)
  mseResults.append(meanFoldMSE)
print(mseResults)

In [ ]:
#Deliverable 2
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
plt.xscale('log')
print(Tuning_parameters)
print(mseResults)
plt.plot(Tuning_parameters, mseResults)
ax.set_ylabel("MSE")
ax.set_xlabel('\u03BB')
ax.set_title(' 5 CV Tuning Parameters')
plt.show()

In [ ]:
#Deliverable 3: Indicate the value of 𝜆 that generated the smallest CV(5) error. 
minCVError = min(mseResults)
minCVIndex = mseResults.index(minCVError)
𝜆 = Tuning_parameters[minCVIndex]
print(minCVError)
print("optimal lamda")
𝜆

In [ ]:
#Deliverable 4
optimalLamda = 𝜆
lr = 10**-5
iteration = 10**4
verbose = 1
coef = []
tX = np.array(X)
tY = np.array(y)

ridge = RidgeRegression(lr,iteration,optimalLamda,verbose)
ridge.fit(tX,tY)
coef = ridge.W
print("Best Params Estimate")
print(coef)

In [ ]:
### Collecting test data



featureExtract = pd.read_csv("TestData_N111_p10.csv",usecols = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"])

quantitative = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"]


for featureName in featureExtract:
  featureCol = []
  for featureValues in featureExtract[featureName]:
    featureCol.append(featureValues)
  Features.append(featureCol)

#Feature Normalization and Scaling
ScaledFeatures = []
for featureType in Features:
  print(featureType)
  scaledFeatureType = []
  mean = np.mean(featureType)
  std = np.std(featureType)
  #Scale each feature value
  for val in featureType:
    newVal = (val - mean) / std
    scaledFeatureType.append(newVal)
  ScaledFeatures.append(scaledFeatureType)
#Labels in list.
for label in df['Ancestry']:
  Labels.append(label)


print("Features")
print(Features)
print("Scaled Features")
print(np.array(ScaledFeatures).shape)
print("Training Data")
print(featureExtract)

X = [[ScaledFeatures[0][i]] + [ScaledFeatures[1][i]] + [ScaledFeatures[2][i]] + [ScaledFeatures[3][i]] + [ScaledFeatures[4][i]] + [ScaledFeatures[5][i]] + [ScaledFeatures[6][i]] + [ScaledFeatures[7][i]] + [ScaledFeatures[8][i]] + [ScaledFeatures[9][i] ]for i in range(len(ScaledFeatures[0]))]

F = np.array(X)
print(F.shape)
print(X[0])
# Centering labels
#Labels -= np.mean(Labels)
y = Labels

#𝐾 =5 
#continental  ancestries  (African,  European,  East  Asian,  Oceanian,  or  Native  American).
cat = ["African","European","EastAsian", "Oceanian", "NativeAmerican"] 
def one_hot_encoding(labels,categories):
  newArray = []
  for item in labels:
    tmp = np.zeros(len(categories))
    for x in range(len(categories)):
      if item == categories[x]:
        tmp[x] = 1
    newArray.append(tmp)
  return newArray
y = one_hot_encoding(y,cat)
print(np.array(y).shape)
#print(df)
#printing the first instance of credit data

In [ ]:
### Making predictions for all of the classes
predictions = ridge.predict(np.array(X))
print(predictions)

In [ ]:
#Deliverable 5
# The unknown seem to be predicted as majority african and mexican ancestry. 

In [ ]:
!jupyter nbconvert --to html Assignment3.ipynb

In [ ]:
# I tried to implement this assignment correctly, however I think I did not do it correctly and may have implemented a new type of neural network or something
# because I am certain this is not the correct implementation probably. 